# Assignment:

## Find all inconsistencies in client_providers.json dataset.

(Goal is to understand and verify data)

In [1]:
import pandas as pd
import json
from pandas.io.json import json_normalize

## EDA

In [2]:
#Customer provided dataset
client_providers = pd.read_json('client_providers.json', lines = True)
client_providers.head()

,druid,npi,f_name,l_name,titles,specialties
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,[md],[rheumatologist]
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"[pt, md]","[family-practitioner, physical-therapist]"
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"[pt, md]","[family-practitioner, physical-therapist]"
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,[pt],[physical-therapist]
4,230dc410045f63c44f43e05ed799e2f4,1710010830,grisele,pastrana rodriguez,[md],[family-practitioner]


In [3]:
client_providers.shape

(10050, 6)

In [4]:
#American Board of Medical Specialties (ABMS)/External Dataset
abms = pd.read_json('abms.json', lines = True)
abms.head()

,npi,f_name,l_name,date_of_birth
0,1356659510,maryangely,moreno campa,1979-12-12
1,1104806892,raymond,pumarejo,1961-01-19
2,1225324825,nicholas,yagoda,1976-12-24
3,1285892877,nicholas,crosby,1978-11-07
4,1710956974,richard,mccurdy,1946-10-25


In [5]:
abms.shape

(9945, 4)

In [6]:
#Attestation Campaign/Internal Data (after calling MD's directly to confirm/update data)
attested_providers = pd.read_json('attested_providers.json', lines = True)
attested_providers.head()

,druid,npi,f_name,l_name,titles,specialties
0,2b44486e322366dd27f1a62acdbae01e,1497936900,andrew,barleben,[md],[vascular-surgeon]
1,f54129a903ec1260e9139cb99b752360,1538484167,howard,bowers,NaN,NaN
2,e4f18a4d71b81f7c32b0ee1200377554,1851813695,katherine,nataro,[arnp],[nurse-practitioner]
3,0f4be2efb6051b60a3e784b52448b0b7,1730277054,kamalinee,deshpande,[md],"[general-practitioner, family-practitioner]"
4,aaf4aa01c834714643d83e55b8e63cd2,1578759023,diogenes,duarte,[md],"[sleep-medicine-doctor, pulmonologist]"


In [7]:
attested_providers.shape

(9482, 6)

## Checking for NaN's

In [8]:
client_providers.isnull().sum()

druid           0
npi             0
f_name          0
l_name          0
titles         34
specialties     0
dtype: int64

In [9]:
abms.isnull().sum()

npi              0
f_name           0
l_name           0
date_of_birth    0
dtype: int64

In [10]:
attested_providers.isnull().sum()

druid             0
npi               0
f_name            0
l_name            0
titles         1353
specialties    1241
dtype: int64

The null's from client_providers and attested_providers will be dropped.

## Checking for duplicates.

In [11]:
abms.duplicated().sum()

0

There needs to be a little more work for client_providers and attested_providers since they have lists, which are not accepted by duplicated().

In [12]:
client_providers.loc[client_providers['npi'] == 1053349886]

,druid,npi,f_name,l_name,titles,specialties
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,[md],[rheumatologist]


In [13]:
#As a reminder, what we are handling
client_providers['titles'].isnull().sum()

34

In [14]:
#zooming in:
client_providers['titles'][70:74]

70      [md]
71       NaN
72    [lmft]
73      [md]
Name: titles, dtype: object

Decision made to dropna since 34/10050 rows is only .3% of the dataset.

In [15]:
client_providers = client_providers.dropna()

In [16]:
#Verified that NaN's were dropped.
client_providers['titles'][70:74]

70      [md]
72    [lmft]
73      [md]
74      [md]
Name: titles, dtype: object

### Now, onto changing lists into string to check df's for duplicates.

In [17]:
client_providers['titles'] = client_providers['titles'].apply(', '.join)

In [18]:
client_providers.head()

,druid,npi,f_name,l_name,titles,specialties
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,[rheumatologist]
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","[family-practitioner, physical-therapist]"
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","[family-practitioner, physical-therapist]"
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,[physical-therapist]
4,230dc410045f63c44f43e05ed799e2f4,1710010830,grisele,pastrana rodriguez,md,[family-practitioner]


In [19]:
client_providers['specialties'] = client_providers['specialties'].apply(', '.join)

In [20]:
client_providers.head()

,druid,npi,f_name,l_name,titles,specialties
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist"
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist"
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist
4,230dc410045f63c44f43e05ed799e2f4,1710010830,grisele,pastrana rodriguez,md,family-practitioner


In [21]:
#Keep in mind, this output is after 3% of the data is taken out. So within the 3% there could have been duplicates as well.
#However, due to the limitation of duplicated() function, had to eliminate NaN's before determining duplicates in (remaining) dataset. 
client_providers.duplicated().sum()

0

Now, on to the last table attested_providers. There is a greater percentage of nulls. As a reminder, there are 1353 nulls (14% of dataset) in titles column and 1241 (13% of dataset) nulls in specialties column.

This is a huge number compared to 3% of dataset that was eliminated from client_providers. However, at least for 86% of the data, we can see how many duplicates there are.

In [22]:
#As a reminder, what we are handling
attested_providers['titles'].isnull().sum(), attested_providers['specialties'].isnull().sum()

(1353, 1241)

In [23]:
attested_providers['titles'][:10]

0      [md]
1       NaN
2    [arnp]
3      [md]
4      [md]
5      [md]
6      [md]
7      [md]
8       NaN
9      [md]
Name: titles, dtype: object

In [24]:
attested_providers['specialties'][:10]

0                             [vascular-surgeon]
1                                            NaN
2                           [nurse-practitioner]
3    [general-practitioner, family-practitioner]
4         [sleep-medicine-doctor, pulmonologist]
5    [interventional-cardiologist, cardiologist]
6                    [infectious-disease-doctor]
7                          [family-practitioner]
8                                            NaN
9                                    [internist]
Name: specialties, dtype: object

In [25]:
attested_providers = attested_providers.dropna()

In [26]:
#Check to see if NaN's dropped from titles column
attested_providers['titles'][:10]

0         [md]
2       [arnp]
3         [md]
4         [md]
5         [md]
6         [md]
7         [md]
9         [md]
10        [md]
11    [md, od]
Name: titles, dtype: object

In [27]:
#Check to see if NaN's dropped from specialties column
attested_providers['specialties'][:10]

0                              [vascular-surgeon]
2                            [nurse-practitioner]
3     [general-practitioner, family-practitioner]
4          [sleep-medicine-doctor, pulmonologist]
5     [interventional-cardiologist, cardiologist]
6                     [infectious-disease-doctor]
7                           [family-practitioner]
9                                     [internist]
10                                 [psychiatrist]
11                              [ophthalmologist]
Name: specialties, dtype: object

Yes! In the clear =)

In [28]:
attested_providers['titles'] = attested_providers['titles'].apply(', '.join)

In [29]:
attested_providers.head()

,druid,npi,f_name,l_name,titles,specialties
0,2b44486e322366dd27f1a62acdbae01e,1497936900,andrew,barleben,md,[vascular-surgeon]
2,e4f18a4d71b81f7c32b0ee1200377554,1851813695,katherine,nataro,arnp,[nurse-practitioner]
3,0f4be2efb6051b60a3e784b52448b0b7,1730277054,kamalinee,deshpande,md,"[general-practitioner, family-practitioner]"
4,aaf4aa01c834714643d83e55b8e63cd2,1578759023,diogenes,duarte,md,"[sleep-medicine-doctor, pulmonologist]"
5,699760b1ba7cd7db7f27d0e2c70e1e62,1316992175,thiruvallur,vallabhan,md,"[interventional-cardiologist, cardiologist]"


In [30]:
attested_providers['specialties'] = attested_providers['specialties'].apply(', '.join)

In [31]:
attested_providers.head()

,druid,npi,f_name,l_name,titles,specialties
0,2b44486e322366dd27f1a62acdbae01e,1497936900,andrew,barleben,md,vascular-surgeon
2,e4f18a4d71b81f7c32b0ee1200377554,1851813695,katherine,nataro,arnp,nurse-practitioner
3,0f4be2efb6051b60a3e784b52448b0b7,1730277054,kamalinee,deshpande,md,"general-practitioner, family-practitioner"
4,aaf4aa01c834714643d83e55b8e63cd2,1578759023,diogenes,duarte,md,"sleep-medicine-doctor, pulmonologist"
5,699760b1ba7cd7db7f27d0e2c70e1e62,1316992175,thiruvallur,vallabhan,md,"interventional-cardiologist, cardiologist"


In [32]:
attested_providers.duplicated().sum()

0

Need to keep in mind that this is the case after 14% of the data was dropped. Still could have duplicates in the 14%. However, due to limitation of duplicates() function with NaN's, only dataset without null's could be examined for duplicates. 

# To clarify, not working with full data (NaN's not included) to see if abms or attested_providers does provide validation.

## More EDA

### Testing to see how MD info between diff DF's look like.

In [33]:
client_providers.loc[client_providers['npi'] == 1356659510]

,druid,npi,f_name,l_name,titles,specialties
1602,f14621d86f2c6960232f4aee3033191a,1356659510,maryangely,moreno campa,md,internist
1603,f14621d86f2c6960232f4aee3033191a,1356659510,marangely,moreno campa,md,internist
1604,f14621d86f2c6960232f4aee3033191a,1356659510,maryangely,moreno campa,md,"internist, oncologist"


In [34]:
abms.loc[abms['npi'] == 1356659510]

,npi,f_name,l_name,date_of_birth
0,1356659510,maryangely,moreno campa,1979-12-12


In [35]:
attested_providers.loc[attested_providers['npi'] == 1356659510]

,druid,npi,f_name,l_name,titles,specialties


Search for maryangely by druid if npi not present just to make sure this MD is not listed (druid assigned if npi not assigned).

In [36]:
attested_providers.loc[attested_providers['druid'] == 'f14621d86f2c6960232f4aee3033191a']

,druid,npi,f_name,l_name,titles,specialties


We can see from this initial comparison that MD maryangely moreno campa is listed 3x in client_providers, 1x in abms and 0 times in attested_providers.

With this small sample comparison can already see that there is data integrity issues: maryangely is not spelled like others ( line 1603 in client_providers df).

### Reminder that the goal is to find **inconsistencies**, which come in the following forms:
    a. duplicates (same MD per row though recorded in multiple forms due to misspellings/various titles, specialities)
    b. incomplete (NaN's in client_providers and attested_providers)
    c. more current information (abms and attested_providers has more current info than client_providrs)

# 1. Start with client_providers compared to abms.

In [37]:
c_a_inner = pd.merge(client_providers, abms, on='npi', how='inner')
c_a_inner.head()

,druid,npi,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,max,harris,1952-02-10
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",abdelmajid,sabour,1964-05-04
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",abdelmajid,sabour,1964-05-04
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,abdelmajid,sabour,1964-05-04
4,230dc410045f63c44f43e05ed799e2f4,1710010830,grisele,pastrana rodriguez,md,family-practitioner,griselle,pastrana,1969-04-30


In [38]:
c_a_inner.shape

(6863, 9)

Only **6863** rows in common between client_providers and abms. This means that there will be this many rows from client_providers to check against abms. 

Let us take a look at how MD maryangely info changed after this table transformation.

In [39]:
c_a_inner.loc[c_a_inner['npi'] == 1356659510]

,druid,npi,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth
1122,f14621d86f2c6960232f4aee3033191a,1356659510,maryangely,moreno campa,md,internist,maryangely,moreno campa,1979-12-12
1123,f14621d86f2c6960232f4aee3033191a,1356659510,marangely,moreno campa,md,internist,maryangely,moreno campa,1979-12-12
1124,f14621d86f2c6960232f4aee3033191a,1356659510,maryangely,moreno campa,md,"internist, oncologist",maryangely,moreno campa,1979-12-12


Here, we see that the same MD maryangely has 3 separate entries due to mispelled first name and inconsistent specialty entry (internist and internist/oncologist). However, given that abms is a reference table, lines 1122 and 1124 are the most current. However, that is as far as the verification with abms goes since it does not provide titles/specialties data. (This is where attested_providers table will come in.)

Let's check a few more to get a glimpse at how pervasive this issue may be.

In [40]:
c_a_inner.loc[c_a_inner['npi'] == 1104806892]

,druid,npi,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth
6367,96fb3432ddc5408ec68a627b258cc871,1104806892,raymond,pumarejo,md,"infectious-disease-doctor, pulmonologist",raymond,pumarejo,1961-01-19


There is no problem here. Let us try one more.

In [41]:
c_a_inner.loc[c_a_inner['npi'] == 1225324825] 

,druid,npi,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth
2003,c1a0d0ef0d46394d8a5a2260fb7cebf0,1225324825,nicholas,yagoda,md,family-practitioner,nicholas,yagoda,1976-12-24
2004,c1a0d0ef0d46394d8a5a2260fb7cebf0,1225324825,padma,reddy,md,family-practitioner,nicholas,yagoda,1976-12-24


Interesting. The same npi is assigned to 2 different MD's unless nicholas yagoda has an alias.

To get a peak at how pervasive the problem is will do a groupby npi to see how many multiple entries there are.

In [42]:
groups1_count = c_a_inner.groupby(['npi']).count()
groups1_count.sort_values(['f_name_x'],ascending = False).head(10)

,druid,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth
npi,,,,,,,,
1427223114,5,5,5,5,5,5,5,5
1629331772,4,4,4,4,4,4,4,4
1942200407,4,4,4,4,4,4,4,4
1568758027,4,4,4,4,4,4,4,4
1285901140,4,4,4,4,4,4,4,4
1497749022,4,4,4,4,4,4,4,4
1033436704,4,4,4,4,4,4,4,4
1801834171,4,4,4,4,4,4,4,4
1407965668,4,4,4,4,4,4,4,4


In [43]:
groups1_count.sort_values(['f_name_x'],ascending = False).tail(10)

,druid,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth
npi,,,,,,,,
1366531329,1,1,1,1,1,1,1,1
1366528838,1,1,1,1,1,1,1,1
1366517880,1,1,1,1,1,1,1,1
1366496226,1,1,1,1,1,1,1,1
1366494155,1,1,1,1,1,1,1,1
1366493439,1,1,1,1,1,1,1,1
1366491821,1,1,1,1,1,1,1,1
1366486359,1,1,1,1,1,1,1,1
1366483356,1,1,1,1,1,1,1,1


Here, we see that the most duplicated record is the MD with npi = 1427223114 while there are numerous repeats that are in the 4 range and less. At the same time, there are records that are not repeated such as evident with tail output right above.

## Putting numbers to inconsistencies.   

### For this join, need to check if client_providers' first_name and last_name are the same in abms. 

### There can be two different outcomes:

    a. match/equal (T)
    b. more current/updated info in abms table (F)

Percentage of T and F will be determined. Goal is to have more T than F occurrences. 

In [44]:
c_a_inner['f_names_eq'] = c_a_inner['f_name_x'] == c_a_inner['f_name_y']

In [45]:
c_a_inner.head()

,druid,npi,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth,f_names_eq
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,max,harris,1952-02-10,True
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",abdelmajid,sabour,1964-05-04,False
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",abdelmajid,sabour,1964-05-04,True
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,abdelmajid,sabour,1964-05-04,False
4,230dc410045f63c44f43e05ed799e2f4,1710010830,grisele,pastrana rodriguez,md,family-practitioner,griselle,pastrana,1969-04-30,False


In [46]:
c_a_inner.f_names_eq.value_counts()

True     5701
False    1162
Name: f_names_eq, dtype: int64

### Percentage of f_name_eq True and Percentage of False:
**83%** for True and 17% for False

In [47]:
c_a_inner['l_names_eq'] = c_a_inner['l_name_x'] == c_a_inner['l_name_y']

In [48]:
c_a_inner.head()

,druid,npi,f_name_x,l_name_x,titles,specialties,f_name_y,l_name_y,date_of_birth,f_names_eq,l_names_eq
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,max,harris,1952-02-10,True,True
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",abdelmajid,sabour,1964-05-04,False,False
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",abdelmajid,sabour,1964-05-04,True,True
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,abdelmajid,sabour,1964-05-04,False,False
4,230dc410045f63c44f43e05ed799e2f4,1710010830,grisele,pastrana rodriguez,md,family-practitioner,griselle,pastrana,1969-04-30,False,False


In [49]:
c_a_inner.l_names_eq.value_counts()

True     5668
False    1195
Name: l_names_eq, dtype: int64

### Percentage of l_name_eq True and Percentage of False:
**83%** for True and 17% for False

# 2. Let us now compare client_providers to attested_providers table and see if there are any better results given that this table has exact columns to client_providers.

In [50]:
c_att_inner = pd.merge(client_providers, attested_providers, on='npi', how='inner')
c_att_inner.head()

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,09a3d8f53bd041d5a244a37c37566efa,max,harris,md,rheumatologist
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist
4,496edcfabc3ffda82d83ad27f948cd2b,1790784189,joan,fambrough,crna,certified-registered-nurse-anesthetist,496edcfabc3ffda82d83ad27f948cd2b,joan,lee,crna,"certified-registered-nurse-anesthetist, certif..."


In [51]:
c_att_inner.shape

(6709, 11)

**6709** rows in common between client_providers and attested_providers. This is less than result from client_providers joined to abms.

Let's see how the same npi's turn out in this new combination.

In [52]:
c_att_inner.loc[c_att_inner['npi'] == 1356659510]

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y


This empty result means that this npi was not attested. Let us try the other two npi's. 

In [53]:
c_att_inner.loc[c_att_inner['npi'] == 1104806892]

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y
6198,96fb3432ddc5408ec68a627b258cc871,1104806892,raymond,pumarejo,md,"infectious-disease-doctor, pulmonologist",96fb3432ddc5408ec68a627b258cc871,raymond,pumarejo,md,"infectious-disease-doctor, critical-care-docto..."


Yay! This npi existed after join between client_providers and attested_providers. Here, we can see once again close, but not exact specialties description.

In [54]:
c_att_inner.loc[c_att_inner['npi'] == 1225324825] 

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y
1956,c1a0d0ef0d46394d8a5a2260fb7cebf0,1225324825,nicholas,yagoda,md,family-practitioner,c1a0d0ef0d46394d8a5a2260fb7cebf0,nicholas,yagoda,md,family-practitioner
1957,c1a0d0ef0d46394d8a5a2260fb7cebf0,1225324825,padma,reddy,md,family-practitioner,c1a0d0ef0d46394d8a5a2260fb7cebf0,nicholas,yagoda,md,family-practitioner


Just like the first comparison, two different names show up for this npi search. Though everything else is consistent (titles and specialties), which is good.

To get a peak at how pervasive the problem is will do a groupby npi to see how many multiple entries there are.

In [55]:
groups2_count = c_att_inner.groupby(['npi']).count()
groups2_count.sort_values(['f_name_x'],ascending = False).head(10)

,druid_x,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y
npi,,,,,,,,,,
1558300269,4,4,4,4,4,4,4,4,4,4
1407965668,4,4,4,4,4,4,4,4,4,4
1619155546,4,4,4,4,4,4,4,4,4,4
1497749022,4,4,4,4,4,4,4,4,4,4
1629004957,4,4,4,4,4,4,4,4,4,4
1285901140,4,4,4,4,4,4,4,4,4,4
1033436704,4,4,4,4,4,4,4,4,4,4
1467594184,4,4,4,4,4,4,4,4,4,4
1629331772,4,4,4,4,4,4,4,4,4,4


In [56]:
groups2_count.sort_values(['f_name_x'],ascending = False).tail(10)

,druid_x,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y
npi,,,,,,,,,,
1366760027,1,1,1,1,1,1,1,1,1,1
1366580631,1,1,1,1,1,1,1,1,1,1
1366717605,1,1,1,1,1,1,1,1,1,1
1366717159,1,1,1,1,1,1,1,1,1,1
1366659260,1,1,1,1,1,1,1,1,1,1
1366622466,1,1,1,1,1,1,1,1,1,1
1366605495,1,1,1,1,1,1,1,1,1,1
1366593576,1,1,1,1,1,1,1,1,1,1
1366584674,1,1,1,1,1,1,1,1,1,1


Here, we see that many npi records are in the 4 range and less. At the same time, there are records that are not repeated such as evident with tail output right above.

## Putting numbers to inconsistencies again.   

### For this join, need to check if client_providers' first_name, last_name, titles and specialties are the same in attested_providers. 

There can be two different outcomes:

    a. match/equal (T)
    b. more current/updated info in attested_providers table (F)

Percentage of T and F will be determined. Goal is to have more T than F occurrences.

In [57]:
c_att_inner['f_names_eq'] = c_att_inner['f_name_x'] == c_att_inner['f_name_y']

In [58]:
c_att_inner.head()

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y,f_names_eq
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,09a3d8f53bd041d5a244a37c37566efa,max,harris,md,rheumatologist,True
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,False
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True
4,496edcfabc3ffda82d83ad27f948cd2b,1790784189,joan,fambrough,crna,certified-registered-nurse-anesthetist,496edcfabc3ffda82d83ad27f948cd2b,joan,lee,crna,"certified-registered-nurse-anesthetist, certif...",True


In [59]:
c_att_inner.f_names_eq.value_counts()

True     6249
False     460
Name: f_names_eq, dtype: int64

### Percentage of f_name_eq True and Percentage of False:
**93%** for True and 7% for False

In [60]:
c_att_inner['l_names_eq'] = c_att_inner['l_name_x'] == c_att_inner['l_name_y']

In [61]:
c_att_inner.head()

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y,f_names_eq,l_names_eq
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,09a3d8f53bd041d5a244a37c37566efa,max,harris,md,rheumatologist,True,True
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True,True
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,False,False
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True,True
4,496edcfabc3ffda82d83ad27f948cd2b,1790784189,joan,fambrough,crna,certified-registered-nurse-anesthetist,496edcfabc3ffda82d83ad27f948cd2b,joan,lee,crna,"certified-registered-nurse-anesthetist, certif...",True,False


In [62]:
c_att_inner.l_names_eq.value_counts()

True     6150
False     559
Name: l_names_eq, dtype: int64

### Percentage of l_name_eq True and Percentage of False:
**92%** for True and 8% for False

In [63]:
c_att_inner['titles_eq'] = c_att_inner['titles_x'] == c_att_inner['titles_y']

In [64]:
c_att_inner.head()

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y,f_names_eq,l_names_eq,titles_eq
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,09a3d8f53bd041d5a244a37c37566efa,max,harris,md,rheumatologist,True,True,True
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True,True,False
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,False,False,False
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True,True,True
4,496edcfabc3ffda82d83ad27f948cd2b,1790784189,joan,fambrough,crna,certified-registered-nurse-anesthetist,496edcfabc3ffda82d83ad27f948cd2b,joan,lee,crna,"certified-registered-nurse-anesthetist, certif...",True,False,True


In [65]:
c_att_inner.titles_eq.value_counts()

True     6163
False     546
Name: titles_eq, dtype: int64

### Percentage of titles_eq True and Percentage of False:
**92%** for True and 8% for False

In [66]:
c_att_inner['specialties_eq'] = c_att_inner['specialties_x'] == c_att_inner['specialties_y']

In [67]:
c_att_inner.head()

,druid_x,npi,f_name_x,l_name_x,titles_x,specialties_x,druid_y,f_name_y,l_name_y,titles_y,specialties_y,f_names_eq,l_names_eq,titles_eq,specialties_eq
0,09a3d8f53bd041d5a244a37c37566efa,1053349886,max,harris,md,rheumatologist,09a3d8f53bd041d5a244a37c37566efa,max,harris,md,rheumatologist,True,True,True,True
1,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True,True,False,False
2,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,abdelmajid,sabour,"pt, md","family-practitioner, physical-therapist",19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,False,False,False,False
3,19a73029b597b48fa7d7d6631a1a5c9e,1942283189,krista,sabo,pt,physical-therapist,19a73029b597b48fa7d7d6631a1a5c9e,krista,sabo,pt,physical-therapist,True,True,True,True
4,496edcfabc3ffda82d83ad27f948cd2b,1790784189,joan,fambrough,crna,certified-registered-nurse-anesthetist,496edcfabc3ffda82d83ad27f948cd2b,joan,lee,crna,"certified-registered-nurse-anesthetist, certif...",True,False,True,False


In [68]:
c_att_inner.specialties_eq.value_counts()

True     4501
False    2208
Name: specialties_eq, dtype: int64

### Percentage of specialties_eq True and Percentage of False:
**67%** for True and 33% for False

# Conclusions:

### Inconsistencies found per npi / (MD) were:

    a. misspelled name
    b. varying titles
    c. varying specialties
    d. npi assigned to multiple MD's
    e. NaN's/missing information
    f. more current info in abms/attested providers

For the most part, True was between 80% and 90%. Only when it came to specialties there was not a match between client_providers dataset and attested_providers. This explains the percentage drop to lower than 80%. Given NaN's were dropped prior to join to attested_providers, client_providers had inaccurate information. 

In the end, client_providers table had mostly consistent data except for when it came to npi specialties data. 

This leads to a more developed method for records accuracy checking to report to clients when given their dataset.

## BONUS:

 _(planned for multiple truth sources though assignment asks to just plan for one - attested_providers)_

Methodology: 

1. Internal analysis: Are there npi duplicates? If so, what does sampling some of them reveal? (E.g. various first names/last names/titles/specialties/some of these cases/all of the above?)

2. Comparison between one truth source, column-by-column.

3. Comparison with another/multiple truth source(s), column-by-column to get average truth.

Metrics: 

1. Duplicates: find % of whole dataset

2. One Verification: for each column, find percent True, report overall True

3. Multiple Verification: for each column re-verified, average columns of True, get overall True

Justification for My Approach:
In order to get most current data to provide Quest Analytics customers most current MD availability, need to check if their own data has integrity issues such as duplicate npi/MD's. On top of that, need to check to see if this data is correct by verifying across truth source(s).

## Results from provided data that could be used by Quest Analytics' clients:

### 1. client_providers Duplicates:

In [69]:
c_duplicates = client_providers.groupby(['npi']).count()

In [70]:
c_duplicates[c_duplicates > 1.0].count()

druid          1369
f_name         1369
l_name         1369
titles         1369
specialties    1369
dtype: int64

Out of the original dataframe minus NaN's (10016 entries), **1369** have more than 1 copy of itself (duplicates). This is 14% of the data, which is a huge amount. 

### 2. One Verification (including duplicates):
-Against abms only-

83% for both f_name and l_name. So overall True average is: 83%

-Against attested_providers only-
93% for f_name, 92% for l_name, 92% for titles, 67% for specialties. So overall True average is: 86%

### 3. Multiple Verification (including duplicates): 
    only columns that are reverified are: f_name and l_name
        Average of f_name: (83+93)/2 = 88%
        Average of l_name: (83+92)/2 = 87.5%
        Overall average: 87.75%

For the client who provided client_providers table, this would be the estimate of how accurate the original data is, compared to multiple sources of truth (abms and attested_providers).